## SAM Sequence Nuclei Detection, Quantification and Alignment

This script provides the code necessary to run nuclei detection, signal quantification, SAM alignment and organ primordia detection on **one** time-lapse sequence of confoocal microscopy acquisitions of a Shoot Apical Meristem. It requires several things to be fully functional :
* The images should be either in `.tif`, `.czi` or `.lsm` format. In the case of `.tif` prefer the [ImageJ encoding](https://imagej.net/TIFF)
* The `.csv` configuration files and the microscopy directory should be prepared as [detailed in the documentation](https://mosaic.gitlabpages.inria.fr/publications/sam_spaghetti/scripts/file_architecture.html)
* The images should be multichannel acquisitions with channel names given in an `experiment_info.csv` file:
    * One channel providing a reference for nuclei (should also be the `reference_name`)
    * One channel corresponding to Auxin bio-sensor DII-Venus (should be named **DIIV**)
    * One channel corresponding to the CLV3 peptide repoorter (should be named **CLV3**)

> Note: as it is designed to work on the stochiometric reporter **qDII**, the script will compute a ratio of quantified levels of **DIIV** and `reference_name` to estimate Auxin levels.


### Library imports

The `sam_spaghetti` library comes with a set of high level functions that generally process a time-lapse SAM sequence, identified by a unique `sequence_name` nomenclature. As the data processed by the pipeline can be voluminous (images, transforms,...) those functions rely a lot on reading from and writing to the disk, hence filling a substantial space on your hard drive.

In [ ]:
import logging
import argparse
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sam_spaghetti
from sam_spaghetti.sam_microscopy_loading import load_image_from_microscopy
from sam_spaghetti.sam_sequence_info import get_experiment_name, get_experiment_microscopy, get_nomenclature_name, get_experiment_channels, get_experiment_reference, get_sequence_orientation, get_experiment_microscope_orientation, update_lut_ranges
from sam_spaghetti.detection_quantification import detect_and_quantify
from sam_spaghetti.sam_sequence_loading import load_sequence_signal_images, load_sequence_signal_image_slices, load_sequence_signal_data
from sam_spaghetti.signal_image_slices import sequence_signal_image_slices, sequence_image_primordium_slices, sequence_signal_data_primordium_slices
from sam_spaghetti.signal_image_plot import signal_image_plot, signal_nuclei_plot, signal_map_plot, signal_image_all_primordia_plot, signal_nuclei_all_primordia_plot, signal_map_all_primordia_plot
from sam_spaghetti.signal_map_computation import compute_signal_maps, compute_primordia_signal_maps, compute_average_signal_maps, compute_average_primordia_signal_maps
from sam_spaghetti.sequence_image_registration import register_sequence_images, apply_sequence_registration
from sam_spaghetti.signal_data_compilation import compile_signal_data, compile_primordia_data
from sam_spaghetti.sequence_growth_estimation import compute_growth
from sam_spaghetti.sam_sequence_primordia_alignment import align_sam_sequence, detect_organ_primordia
from sam_spaghetti.data_visualization import primordium_data_plot

logging.getLogger().setLevel(logging.ERROR)

%matplotlib inline

### Parameters

It is essential for the `sam_spaghetti` tools that files providing information on the data can be found in the directory specified by the parameter `data_dirname`, and that folders containing microscopy image files (referenced in those configuration files) can be found in the directory specified by the parameter `microscopy_dirname`. 

By default, the `share/data/` directory of the `sam_spaghetti` package is used, and a new folder called `nuclei_images` will be created to store the outputs of the script. This directory can be changed in the `image_dirname` parameter.

In [ ]:
data_dirname = sam_spaghetti.__path__[0]+"/../../share/data"
microscopy_dirname = data_dirname+"/microscopy"
image_dirname = data_dirname+"/nuclei_images"

Unlike the command-line scripts provided by the package, this example is dedicated to the processing of a single time-lapse sequence of a shoot apical meristem. This sequence is identified by its `experiment` identifier (which should be present in the `experiment_info.csv` configuration file) and by a SAM number `sam_id` (which should be found in both `nomenclature.csv` and `nuclei_image_sam_orientation.csv` configuration files)

In [ ]:
experiment = 'E27'
sam_id = 7

The pipeline consists of several processing steps, and since files are saved at each step, it is not necessary to re-run all steps to get to final results. The step parameters can be set from `True` to `False` in that case, unless you specifically want to try different parameter values for a specific step. Note that for the first processing of a new sequence, they should all be turned to `True`.

In [ ]:
detection = True
registration = True
primordia_alignment = True

In [ ]:
resampling_voxelsize = 1.

This script generates several displays of images, quantified intensities and signal maps. Here you can customize what will be displayed (by adding signals to the list, signals that should correspond to `channel_names` in the `experiment_info.csv` file) and how (by choosing between `max_intensity` and `L1_slice` for image visualization for instance; note that `L1_slice` is much more computationally expensive).

Note that lookuptables for the different displayed signals can be customized by editing the `signal_ranges.csv` configuration file.

In [ ]:
signals_to_display = ['DR5']
projection_type = "max_intensity"
update_lut_ranges(data_dirname)

### Microscopy image loading & Nuclei detection

In [ ]:
experiment_name = get_experiment_name(experiment,data_dirname)
if experiment_name == "":
    logging.error("Experiment identifier \""+experiment+"\" not recognized (consider adding it to the experiment_info.csv file in the data directory)")
else:
    if detection and (microscopy_dirname is not None):
        experiment_dirname = microscopy_dirname+"/"+get_experiment_microscopy(experiment,data_dirname)
        if os.path.exists(experiment_dirname+"/RAW"):
            experiment_dirname += "/RAW"

        if not os.path.exists(experiment_dirname):
            logging.warning("Microscopy directory not found for "+experiment+", no detection will be performed.")
        else:                    
            microscopy_filenames = [experiment_dirname+"/"+f for f in os.listdir(experiment_dirname) if np.any([ext in f for ext in ['.czi','.lsm','.tif']])]
            nomenclature_names = [get_nomenclature_name(microscopy_filename,data_dirname) for microscopy_filename in microscopy_filenames]
            microscopy_filenames = [m for m,n in zip(microscopy_filenames,nomenclature_names) if n is not None and "_sam"+str(sam_id).zfill(2) in n]
            
            channel_names = get_experiment_channels(experiment, data_dirname)
            reference_name = get_experiment_reference(experiment, data_dirname)
        
            for microscopy_filename in microscopy_filenames:
                nomenclature_name = get_nomenclature_name(microscopy_filename,data_dirname)

                if nomenclature_name is not None:
                    sequence_name = nomenclature_name[:-4]
                    if not os.path.exists(image_dirname):
                        os.makedirs(image_dirname)
                    if not os.path.exists(image_dirname+"/"+sequence_name):
                        os.makedirs(image_dirname+"/"+sequence_name)
                    if not os.path.exists(image_dirname+"/"+sequence_name+"/"+nomenclature_name):
                        os.makedirs(image_dirname+"/"+sequence_name+"/"+nomenclature_name)

                    if detection:
                        logging.info("--> Running detection on "+nomenclature_name+" "+reference_name)
                        img_dict = load_image_from_microscopy(microscopy_filename, resampling_voxelsize=resampling_voxelsize, save_images=True, image_dirname=image_dirname, nomenclature_name=nomenclature_name, channel_names=channel_names, verbose=False, loglevel=1)
                        detect_and_quantify(img_dict, surface_voxelsize=resampling_voxelsize, reference_name=reference_name, signal_names=channel_names, image_dirname=image_dirname, nomenclature_name=nomenclature_name, verbose=False, loglevel=1)
                else:
                    logging.warning("--> No nomenclature found for " + microscopy_filename + ", skipping...")

### Multichannel image sequence

In [ ]:
sequence_name = experiment_name + "_sam" + str(sam_id).zfill(2)
reference_name = get_experiment_reference(experiment, data_dirname)

logging.info("--> Loading sequence " + sequence_name)

signal_images = load_sequence_signal_images(sequence_name, image_dirname, signal_names=[reference_name]+signals_to_display, verbose=False, loglevel=1)

if len(signal_images)==0:
    logging.error("--> Unable to load sequence " + sequence_name + " (make sure you have run detection first)")
    
microscope_orientation = get_experiment_microscope_orientation(experiment, data_dirname)

logging.info("--> Plotting signal images " + sequence_name)
signal_image_slices = load_sequence_signal_image_slices(sequence_name, image_dirname, projection_type=projection_type, aligned=False, verbose=False, loglevel=1)
if len(signal_image_slices) == 0:
    signal_image_slices = sequence_signal_image_slices(sequence_name, image_dirname, reference_name=reference_name, microscope_orientation=microscope_orientation,
                                                       projection_type=projection_type, resolution=None, aligned=False, save_files=True, verbose=False, loglevel=1)

figure = signal_image_plot({s:signal_image_slices[s] for s in [reference_name,'CLV3','DIIV']+signals_to_display if s in signal_image_slices}, reference_name=reference_name, projection_type=projection_type, resolution=0.25, aligned=False, verbose=False, loglevel=1)

### L1 Nuclei signal quantification

In [ ]:
r_max = list(signal_images[reference_name].values())[0].shape[0] * list(signal_images[reference_name].values())[0].voxelsize[0] / 2.
signal_data = load_sequence_signal_data(sequence_name, image_dirname, normalized=False, aligned=False, verbose=False, loglevel=1)

logging.info("--> Plotting detected nuclei signals " + sequence_name)
figure = signal_nuclei_plot(signal_data, r_max=r_max, signal_names=['CLV3','DIIV']+[s for s in signals_to_display if s in list(signal_data.values())[0].columns], normalized=False, verbose=False, loglevel=1)


### 2D Maps of epidermal Auxin level

In [ ]:
logging.info("--> Compiling signal data")
compile_signal_data([experiment], save_files=True, image_dirname=image_dirname, data_dirname=data_dirname, verbose=False, loglevel=1)
normalized_signal_data = load_sequence_signal_data(sequence_name, image_dirname, normalized=True, aligned=False, verbose=False, loglevel=1)

figure = signal_nuclei_plot(normalized_signal_data, r_max=r_max, signal_names=['Auxin'], normalized=False, verbose=False, loglevel=1)

In [ ]:
signal_maps = compute_signal_maps(normalized_signal_data, signal_names=['Auxin'], normalized=False, polar=False, verbose=False, loglevel=1)
logging.info("--> Plotting maps "+sequence_name)
figure = signal_map_plot(signal_maps, verbose=False, loglevel=1)

### Sequence rigid image registration

In [ ]:
if registration:
    logging.info("--> Sequence image registration "+sequence_name)
    register_sequence_images(sequence_name, microscope_orientation=microscope_orientation, pyramid_lowest_level=3, compute_vectorfield=False, save_files=True, image_dirname=image_dirname, reference_name=reference_name, verbose=False, loglevel=1)
apply_sequence_registration(sequence_name, microscope_orientation=microscope_orientation, save_files=True, image_dirname=image_dirname, reference_name=reference_name, verbose=False, loglevel=1)

registered_signal_image_slices = sequence_signal_image_slices(sequence_name, image_dirname, reference_name=reference_name, microscope_orientation=microscope_orientation, projection_type=projection_type, resolution=None, registered=True, aligned=False, save_files=False, verbose=False, loglevel=1)
figure = signal_image_plot({reference_name:registered_signal_image_slices[reference_name]}, reference_name=reference_name, projection_type=projection_type, resolution=None, aligned=False, verbose=False, loglevel=1)

### Sequence alignment : CZ centering + P0 identification

In [ ]:
logging.info("--> Compiling signal data")
compile_signal_data([experiment], save_files=True, image_dirname=image_dirname, data_dirname=data_dirname, verbose=False, loglevel=1)

if primordia_alignment:
    logging.info("--> Sequence primordia alignment "+sequence_name)
    sam_orientation = get_sequence_orientation(sequence_name,data_dirname)
    sequence_aligned_data = align_sam_sequence(sequence_name, image_dirname, sam_orientation=sam_orientation, save_files=True, verbose=False, loglevel=1)

### Aligned signal maps

In [ ]:
aligned_signal_data = load_sequence_signal_data(sequence_name, image_dirname, normalized=True, aligned=True, verbose=False, loglevel=1)

aligned_signal_maps = compute_signal_maps(aligned_signal_data, signal_names=['CLV3','Auxin'] + [s for s in signals_to_display if s in list(aligned_signal_data.values())[0].columns], normalized=False, aligned=True, polar=False, verbose=False, loglevel=1)
logging.info("--> Plotting aligned maps "+sequence_name)
figure = signal_map_plot(aligned_signal_maps, aligned=True, verbose=False, loglevel=1)

### Detection of organ primordia

In [ ]:
detect_organ_primordia(sequence_name, image_dirname,  cell_radius=5, density_k=0.25, save_files=True, verbose=False, loglevel=1)

In [ ]:
logging.info("--> Compiling signal data and primordia data")
compile_signal_data([experiment],save_files=True, image_dirname=image_dirname, data_dirname=data_dirname, aligned=True, verbose=False, loglevel=1)
sequence_primordia_data = compile_primordia_data([experiment],save_files=True, image_dirname=image_dirname, data_dirname=data_dirname, verbose=False, loglevel=1)

#primordium_range = [0]
primordium_range = [-3,-2,-1,0,1,2,3,4,5]
time_range = [0,5,10]

for i_s,signal_name in enumerate(['radial_distance','Auxin']+signals_to_display):
    if signal_name in sequence_primordia_data.columns:
        figure = plt.figure(i_s)
        figure.clf()
        primordium_data_plot(sequence_primordia_data,figure,signal_name=signal_name,primordium_range=primordium_range,time_range=time_range,data_to_plot='detected_auxin_extrema',figure_size=(18,6))
